In [ ]:
import os

In [ ]:
%cd ..
from src.utils import gen_combined_report, gen_bea_report

## Model finetuned via SL and RL

In [3]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune": (os.path.abspath("notebooks/pg_logs/finetune_rl_01_11_2022_13:41/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

## Model pretained on different amount of synthetic data 

In [4]:
model_dict = {
    "Pretrain (0.5K)": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain (2M)": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain (0.5K) + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_03:11:2022_00:32/"), "model-best"),
    "Pretrain (2M) + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

## Model with different success reward

- 0.9 explore coefficient

In [5]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (100 interval + 0.1 Success)": (os.path.abspath("notebooks/pg_logs/finetune_rl_02_11_2022_13:10/"), "model-last"),
    "Pretrain + RL Fine-Tune (100 interval + 0.5 Success)": (os.path.abspath("notebooks/pg_logs/finetune_rl_02_11_2022_17:24/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different explore coefficient

- +0.1 for success

### Model-Best

In [6]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.1 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_14:31/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_10:20/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.9 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_02_11_2022_13:10/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)


### Model-Last

In [7]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.1 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_14:31/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_10:20/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.9 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_02_11_2022_13:10/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different search methods

- +0.1 for success

| Version | Description |
| ------- | ----------- |
| v0      | Exploit = Sample from Candidate Labels with their logits as probs |
| v1      | Exploit = Use logits.argmax() |
| v2      | Exploit = Use candidate_logits.argmax(). Also insert `$KEEP` in the labels |
| v3      | Exploit = Sample from mean(candidate_probs + lev_dist_probs). Also insert `$KEEP` in the labels |
| v4      | v3 with bugfix related with lev_dist |
| v5      | v0 with `$KEEP` label |
| v6      | v5 with `$UNKONWN` label if no candidate_labels |
| v7      | Search algo always checks all labels. `$KEEP` label if no candidate_labels |
| v8      | v7 but only check all labels for replace edit type |
| v9      | v8 with `$UNKONWN` label if no candidate_labels |


### Model-Best

In [8]:
model_dict = {
    "Pretrain (0.5K)": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (v0 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_10:20/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v1 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_06_11_2022_12:24/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v2 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_06_11_2022_15:43/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v3 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_07_11_2022_18:49/"), "model-best"),
    "Pretrain + RL Fine-Tune (v4 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_00:14/"), "model-best"),
    "Pretrain + RL Fine-Tune (v5 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_11:41/"), "model-best"),
    "Pretrain + RL Fine-Tune (v6 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_15:46/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v0 0.9 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_02_11_2022_13:10/"), "model-best"),
    # "Pretrain + RL Fine-Tune (v1 0.9 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_05_11_2022_14:39/"), "model-best"),
    "Pretrain + RL Fine-Tune (v4 0.75-0.25 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_09_11_2022_12:56/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [9]:
model_dict = {
    "Pretrain (0.5K)": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_09:39/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (v0 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_10:20/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v1 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_06_11_2022_12:24/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v2 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_06_11_2022_15:43/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v3 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_07_11_2022_18:49/"), "model-last"),
    "Pretrain + RL Fine-Tune (v4 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_00:14/"), "model-last"),
    "Pretrain + RL Fine-Tune (v5 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_11:41/"), "model-last"),
    "Pretrain + RL Fine-Tune (v6 0.5 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_15:46/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v0 0.9 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_02_11_2022_13:10/"), "model-last"),
    # "Pretrain + RL Fine-Tune (v1 0.9 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_05_11_2022_14:39/"), "model-last"),
    "Pretrain + RL Fine-Tune (v4 0.75-0.25 explore)": (os.path.abspath("notebooks/pg_logs/finetune_rl_09_11_2022_12:56/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different false negative rewards

- 0.5 explore coefficient

### Model-Best

In [10]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.0 fn reward)": (os.path.abspath("notebooks/pg_logs/finetune_rl_07_11_2022_13:51/"), "model-best"),
    "Pretrain + RL Fine-Tune (-0.1 fn reward)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_10:20/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [11]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.0 fn reward)": (os.path.abspath("notebooks/pg_logs/finetune_rl_07_11_2022_13:51/"), "model-last"),
    "Pretrain + RL Fine-Tune (-0.1 fn reward)": (os.path.abspath("notebooks/pg_logs/finetune_rl_03_11_2022_10:20/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different update interval

- 0.5 explore coefficient
- 0.0 fn reward

### Model-Best

In [12]:
model_dict = {
    "Pretrain + RL Fine-Tune (50 interval)": (os.path.abspath("notebooks/pg_logs/finetune_rl_08_11_2022_12:47/"), "model-best"),
    "Pretrain + RL Fine-Tune (100 interval)": (os.path.abspath("notebooks/pg_logs/finetune_rl_07_11_2022_13:51/"), "model-best"),
    "Pretrain + RL Fine-Tune (500 interval)": (os.path.abspath("notebooks/pg_logs/finetune_rl_08_11_2022_16:39/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [13]:
model_dict = {
    "Pretrain + RL Fine-Tune (50 interval)": (os.path.abspath("notebooks/pg_logs/finetune_rl_08_11_2022_12:47/"), "model-last"),
    "Pretrain + RL Fine-Tune (100 interval)": (os.path.abspath("notebooks/pg_logs/finetune_rl_07_11_2022_13:51/"), "model-last"),
    "Pretrain + RL Fine-Tune (500 interval)": (os.path.abspath("notebooks/pg_logs/finetune_rl_08_11_2022_16:39/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different environment

- 0.5 explore coefficient
- 0.0 fn reward

> Dropout disabled in 3e-6 lr experiment

### Model-Best

In [14]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_15:46/"), "model-best"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_11_11_2022_01:04/"), "model-best"),
    "Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_11_11_2022_08:28/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [15]:
model_dict = {
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_01:11:2022_12:09/"), "model-best"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv0 100k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_10_11_2022_15:46/"), "model-last"),
    "Pretrain + RL Fine-Tune (1e-5 lr LevDistv1 200k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_11_11_2022_01:04/"), "model-last"),
    "Pretrain + RL Fine-Tune (3e-6 lr LevDistv1 200k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_11_11_2022_08:28/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different training time

- Pretrained on 2M sentences
- 0.5 explore coefficient
- 0.0 fn reward

### Model-Best

In [16]:
model_dict = {
    "Pretrain (2.0M)": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain (2.0M) + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain (2.0M) + RL Fine-Tune (100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-best"),
    "Pretrain (2.0M) + RL Fine-Tune (500k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_12_11_2022_11:40/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [17]:
model_dict = {
    "Pretrain (2.0M)": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain (2.0M) + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain (2.0M) + RL Fine-Tune (100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-last"),
    "Pretrain (2.0M) + RL Fine-Tune (500k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_12_11_2022_11:40/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model with different reward scales

- Pretrained on 2M sentences
- 0.5 explore coefficient
- 0.0 fn reward
- Reward = Delta Lev-Dist

### Model-Best

In [18]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (100k, 0.1 Scale)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-best"),
    "Pretrain + RL Fine-Tune (100k, 1.0 Scale)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_13_11_2022_13:19/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [19]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (100k, 0.1 Scale)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-last"),
    "Pretrain + RL Fine-Tune (100k, 1.0 Scale)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_13_11_2022_13:19/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model (2M) with different explore coefficient (new vs old methods)

- 1 for success
- GECLevDistV1
- 1.0 reward scale

| Version | Description |
| ------- | ----------- |
| v0      | Exploit = Sample from Candidate Labels with their logits as probs |
| v1      | Exploit = Use logits.argmax() |
| v2      | Exploit = Use candidate_logits.argmax(). Also insert `$KEEP` in the labels |
| v3      | Exploit = Sample from mean(candidate_probs + lev_dist_probs). Also insert `$KEEP` in the labels |
| v4      | v3 with bugfix related with lev_dist |
| v5      | v0 with `$KEEP` label |
| v6      | v5 with `$UNKONWN` label if no candidate_labels |
| v7      | Search algo always checks all labels. `$KEEP` label if no candidate_labels |
| v8      | v7 but only check all labels for replace edit type |
| v9      | v8 with `$UNKONWN` label if no candidate_labels |


### Model-Best

In [20]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (v6 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_13_11_2022_13:19/"), "model-best"),
    "Pretrain + RL Fine-Tune (v7 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_13_11_2022_17:45/"), "model-best"),
    "Pretrain + RL Fine-Tune (v8 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_14_11_2022_10:47/"), "model-best"),
    "Pretrain + RL Fine-Tune (v9 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_14_11_2022_14:48/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [21]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (v6 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_13_11_2022_13:19/"), "model-last"),
    "Pretrain + RL Fine-Tune (v7 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_13_11_2022_17:45/"), "model-last"),
    "Pretrain + RL Fine-Tune (v8 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_14_11_2022_10:47/"), "model-last"),
    "Pretrain + RL Fine-Tune (v9 0.5 explore)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_14_11_2022_14:48/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model (2M) with different fn penalty

- 1 for success
- GECLevDistV1
- 1.0 reward scale

### Model-Best

In [22]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    # "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.0 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_14_11_2022_14:48/"), "model-best"),
    "Pretrain + RL Fine-Tune (-0.5 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_15_11_2022_11:19/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [23]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    # "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.0 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_14_11_2022_14:48/"), "model-last"),
    "Pretrain + RL Fine-Tune (-0.5 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_15_11_2022_11:19/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model (2M) without unsolvable with different fn penalty

- 1 for success
- GECLevDistV1
- 1.0 reward scale
- 200k training timesteps

### Model-Best

In [24]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore, -0.5 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_16_11_2022_12:07/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore, -2.0 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_18_11_2022_15:00/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore, -2.0 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_19_11_2022_13:25/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

### Model-Last

In [25]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore, -0.5 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_16_11_2022_12:07/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5 explore, -2.0 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_18_11_2022_15:00/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore, -2.0 fn penalty)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_19_11_2022_13:25/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

# Model (2M) with and without unsolvable

- 1 for success
- GECLevDistV1
- 1.0 reward scale
- -0.5 fn penalty

### Model-Best

In [26]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    # "Pretrain + RL Fine-Tune (with unsolvable 100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_15_11_2022_11:19/"), "model-best"),
    "Pretrain + RL Fine-Tune (with unsolvable 200k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_17_11_2022_11:52/"), "model-best"),
    # "Pretrain + RL Fine-Tune (without unsolvable 100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_15_11_2022_15:48/"), "model-best"),
    "Pretrain + RL Fine-Tune (without unsolvable 200k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_16_11_2022_12:07/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [27]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    # "Pretrain (2.0M) + RL Fine-Tune (100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-last"),
    # "Pretrain + RL Fine-Tune (with unsolvable 100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_15_11_2022_11:19/"), "model-last"),
    "Pretrain + RL Fine-Tune (with unsolvable 200k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_17_11_2022_11:52/"), "model-last"),
    # "Pretrain + RL Fine-Tune (without unsolvable 100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_15_11_2022_15:48/"), "model-last"),
    "Pretrain + RL Fine-Tune (without unsolvable 200k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_16_11_2022_12:07/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model (2M) with Focal vs Cross-Entropy Loss
- Without unsolvable
- 1 for success
- -2.0 for FN
- GECLevDistV1
- 1.0 reward scale
- 200k training timesteps

### Model-Best

In [28]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore FL)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_18_11_2022_15:00/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore CE)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore FL)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_19_11_2022_13:25/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore CE)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_16:17/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

### Model-Last

In [29]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore FL)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_18_11_2022_15:00/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5 explore CE)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore FL)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_19_11_2022_13:25/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore CE)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_16:17/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

# Model (2M) with different update interval
- CE loss
- Without unsolvable
- 1 for success
- -2.0 for FN
- GECLevDistV1
- 1.0 reward scale
- 200k training timesteps

### Model-Best

In [30]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore 100 interval)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_22_11_2022_11:52/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore 500 interval)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

### Model-Last

In [31]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore 100 interval)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_22_11_2022_11:52/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5 explore 500 interval)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

# Model (2M) with different learning rate
- CE loss
- Without unsolvable
- 1 for success
- -2.0 for FN
- GECLevDistV1
- 1.0 reward scale
- 200k training timesteps
- 500 update interval

### Model-Best

In [32]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore, 5e-6 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_23_11_2022_01:53/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore, 1e-5 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore CE, 5e-6 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_23_11_2022_07:35/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore CE, 1e-5 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_16:17/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

### Model-Last

In [33]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (0.5 explore, 5e-6 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_23_11_2022_01:53/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5 explore, 1e-5 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore CE, 5e-6 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_23_11_2022_07:35/"), "model-last"),
    "Pretrain + RL Fine-Tune (0.5-0.25 explore CE, 1e-5 lr)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_16:17/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

# Model (2M) with different reward range
- CE loss
- Without unsolvable
- GECLevDistV1
- 200k training timesteps
- 0.5 explore

Type A:
- reward range 1.0 to -2.0
- 500 update interval

Type B:
- reward range 0.1 to -0.1
- 100 update interval

### Model-Best

In [34]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (Type A)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-best"),
    "Pretrain + RL Fine-Tune (Type B)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_23_11_2022_13:54/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

### Model-Last

In [35]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (Type A)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_21_11_2022_23:56/"), "model-last"),
    "Pretrain + RL Fine-Tune (Type B)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_23_11_2022_13:54/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)
bea_df = gen_bea_report(model_dict)

# Model (2M) with different environment
- CE loss
- Without unsolvable
- 200k training timesteps
- 0.5 explore
- 100 update interval

### Model-Best

In [36]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (v1)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_23_11_2022_13:54/"), "model-best"),
    "Pretrain + RL Fine-Tune (v2)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_24_11_2022_00:15/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)
# bea_df = gen_bea_report(model_dict)

### Model-Last

In [37]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (v1)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_23_11_2022_13:54/"), "model-last"),
    "Pretrain + RL Fine-Tune (v2)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_24_11_2022_00:15/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)
# bea_df = gen_bea_report(model_dict)

# Model (2M) with different methods

- 1 for success
- -1.0 for fn
- GECLevDistV1
- 0.1 reward scale
- 100 update interval

| Version | Description |
| ------- | ----------- |
| v0      | Exploit = Sample from Candidate Labels with their logits as probs |
| v1      | Exploit = Use logits.argmax() |
| v2      | Exploit = Use candidate_logits.argmax(). Also insert `$KEEP` in the labels |
| v3      | Exploit = Sample from mean(candidate_probs + lev_dist_probs). Also insert `$KEEP` in the labels |
| v4      | v3 with bugfix related with lev_dist |
| v5      | v0 with `$KEEP` label |
| v6      | v5 with `$UNKONWN` label if no candidate_labels |
| v7      | Search algo always checks all labels. `$KEEP` label if no candidate_labels |
| v8      | v7 but only check all labels for replace edit type |
| v9      | v8 with `$UNKONWN` label if no candidate_labels |
| v10     | v9 without `KEEP` label |


### Model-Best

In [38]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (v9, 1e-5 lr)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_23_11_2022_13:54/"), "model-best"),
    "Pretrain + RL Fine-Tune (v10, 5e-6 lr)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_24_11_2022_13:10/"), "model-best"),
}

combined_df = gen_combined_report(model_dict)

### Model-Last

In [39]:
model_dict = {
    "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain + RL Fine-Tune (v9, 1e-5 lr)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_23_11_2022_13:54/"), "model-last"),
    "Pretrain + RL Fine-Tune (v10, 5e-6 lr)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_24_11_2022_13:10/"), "model-last"),
}

combined_df = gen_combined_report(model_dict)

# Model (2M) latest


## Old
- v10
- 5e-6 lr
- 100 update interval
- -1.0 fn reward
- 0.1 reward scale

## v1
- v11
- 200 update interval
- -0.5 fn reward
- 1.0 reward scale

## v2
- v6
- 200 update interval
- 0.0 fn reward
- 1.0 reward scale
- fixed loss.backward() bug

## v3
- use CE instead of FL

## v4
- 0.99 lambda

### Model-Best

In [22]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.50 explore prob 1e-5 lr new v1)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_27_11_2022_12:56/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.75 explore prob 1e-5 lr new v1)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_26_11_2022_00:42/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_13:03/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.50 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_06:51/"), "model-best"),
    "Pretrain + RL Fine-Tune (1.0 to -1.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_06:12/"), "model-best"),
    "Pretrain + RL Fine-Tune (2.0 to -2.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_18:10/"), "model-best"),
    "Pretrain + RL Fine-Tune (5.0 to -5.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_12:15/"), "model-best"),
    # "Pretrain + RL Fine-Tune (5.0 to +0.0 rewards 0.25 explore prob 5e-6 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_00:46/"), "model-best"),
    "Pretrain + RL Fine-Tune (2.0 to +0.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_05:48/"), "model-best"),
    "Pretrain + RL Fine-Tune (5.0 to +0.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_00:18/"), "model-best"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_12:27/"), "model-best"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.00 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_17:55/"), "model-best"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.25 explore prob 1e-5 lr new v3)": (os.path.abspath("results_new/finetune_rl_01_12_2022_00:47/"), "model-best"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.00 explore prob 1e-5 lr new v3)": (os.path.abspath("results_new/finetune_rl_01_12_2022_06:03/"), "model-best"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.00 explore prob 1e-5 lr new v4)": (os.path.abspath("results_new/finetune_rl_01_12_2022_13:14/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.75 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_01:55/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.50-0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_18:42/"), "model-best"),
    "Pretrain (2.0M) + RL Fine-Tune (100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-best"),
    "Pretrain (2.0M) + RL Fine-Tune (500k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_12_11_2022_11:40/"), "model-best"),
}

combined_df = gen_combined_report(model_dict, include_bea=True)

### Model-Last

In [23]:
model_dict = {
    # "Pretrain": (os.path.abspath("notebooks/sl_logs/pretrain_synthetic_31:10:2022_19:25/"), "model-best"),
    "Pretrain + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    # "Pretrain + RL Fine-Tune (0.50 explore prob 1e-5 lr new v1)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_27_11_2022_12:56/"), "model-last"),
    # "Pretrain + RL Fine-Tune (0.75 explore prob 1e-5 lr new v1)": (os.path.abspath("notebooks/pg_logs_final/finetune_rl_26_11_2022_00:42/"), "model-last"),
    # "Pretrain + RL Fine-Tune (0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_13:03/"), "model-last"),
    # "Pretrain + RL Fine-Tune (0.50 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_06:51/"), "model-last"),
    "Pretrain + RL Fine-Tune (1.0 to -1.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_06:12/"), "model-last"),
    "Pretrain + RL Fine-Tune (2.0 to -2.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_18:10/"), "model-last"),
    "Pretrain + RL Fine-Tune (5.0 to -5.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_12:15/"), "model-last"),
    # "Pretrain + RL Fine-Tune (5.0 to +0.0 rewards 0.25 explore prob 5e-6 lr new v2)": (os.path.abspath("results/finetune_rl_29_11_2022_00:46/"), "model-last"),
    "Pretrain + RL Fine-Tune (2.0 to +0.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_05:48/"), "model-last"),
    "Pretrain + RL Fine-Tune (5.0 to +0.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_00:18/"), "model-last"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_12:27/"), "model-last"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.00 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_30_11_2022_17:55/"), "model-last"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.25 explore prob 1e-5 lr new v3)": (os.path.abspath("results_new/finetune_rl_01_12_2022_00:47/"), "model-last"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.00 explore prob 1e-5 lr new v3)": (os.path.abspath("results_new/finetune_rl_01_12_2022_06:03/"), "model-last"),
    "Pretrain + RL Fine-Tune (10. to +0.0 rewards 0.00 explore prob 1e-5 lr new v4)": (os.path.abspath("results_new/finetune_rl_01_12_2022_13:14/"), "model-last"),
    # "Pretrain + RL Fine-Tune (0.75 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_01:55/"), "model-last"),
    # "Pretrain + RL Fine-Tune (0.50-0.25 explore prob 1e-5 lr new v2)": (os.path.abspath("results/finetune_rl_28_11_2022_18:42/"), "model-last"),
    "Pretrain (2.0M) + RL Fine-Tune (100k)": (os.path.abspath("notebooks/pg_logs_new/finetune_rl_12_11_2022_15:44/"), "model-last"),
    "Pretrain (2.0M) + RL Fine-Tune (500k)": (os.path.abspath("notebooks/pg_logs/finetune_rl_12_11_2022_11:40/"), "model-last"),
}

combined_df = gen_combined_report(model_dict, include_bea=True)

In [29]:
model_dict = {
    "Pretrain (2.0M) + SL Fine-Tune": (os.path.abspath("notebooks/sl_logs/finetune_wi+locness_02:11:2022_23:06/"), "model-best"),
    "Pretrain (2.0M) + RL Fine-Tune (500k old)": (os.path.abspath("notebooks/pg_logs/finetune_rl_12_11_2022_11:40/"), "model-last"),
    "Pretrain (2.0M) + RL Fine-Tune (200k new v2)": (os.path.abspath("results_new/finetune_rl_01_12_2022_06:03/"), "model-last"),
    "Pretrain (2.0M) + RL Fine-Tune (200k new v4)": (os.path.abspath("results_new/finetune_rl_01_12_2022_13:14/"), "model-last"),
}

combined_df = gen_combined_report(model_dict, include_bea=True)

In [1]:
import pandas as pd

In [7]:
data_dict = [
    {
        "Type": "SL Finetuning",
        "Experiment Time [Hours]": 0.5,
    },
    {
        "Type": "RL Finetuning 500k episodes",
        "Experiment Time [Hours]": 12.5,
    },
]
df = pd.DataFrame(data_dict)
df["Number of Seeds"] = 5
df["Number of Hyperparameter"] = 1
df["Total Time [Hours]"] = df["Experiment Time [Hours]"]*df["Number of Seeds"]*df["Number of Hyperparameter"]
print(f"Total Time: {df['Total Time [Hours]'].sum()/24:.2f} days")
df

Total Time: 2.71 days


,Type,Experiment Time [Hours],Number of Seeds,Number of Hyperparameter,Total Time [Hours]
0,SL Finetuning,0.5,5,1,2.5
1,RL Finetuning 500k episodes,12.5,5,1,62.5


In [8]:
data_dict = [
    {
        "Type": "SL Finetuning",
        "Experiment Time [Hours]": 0.5,
    },
    {
        "Type": "RL Finetuning 1M episodes",
        "Experiment Time [Hours]": 25,
    },
]
df = pd.DataFrame(data_dict)
df["Number of Seeds"] = 5
df["Number of Hyperparameter"] = 1
df["Total Time [Hours]"] = df["Experiment Time [Hours]"]*df["Number of Seeds"]*df["Number of Hyperparameter"]
print(f"Total Time: {df['Total Time [Hours]'].sum()/24:.2f} days")
df

Total Time: 5.31 days


,Type,Experiment Time [Hours],Number of Seeds,Number of Hyperparameter,Total Time [Hours]
0,SL Finetuning,0.5,5,1,2.5
1,RL Finetuning 1M episodes,25.0,5,1,125.0
